Create and save distributions for 500 odor entities

In [1]:
import pickle as pkl
import numpy as np

In [3]:
with open('binary_opens.pkl','rb') as readfile:
  # ** x is a dictionary, with idx: [chems]
  x = pkl.load(readfile)

  
# ** distributions is a dict with id: { means: [], covariances: [matrix] }
distributions = {}
for id in x.keys():
  odor = x[id]
  n = len(odor)
  odor = odor.reshape(-1, 1).astype(int)

  odor_distribution = {}
  mean = [np.random.random() * 10 + 30 if chem else 0 for chem in odor]
  odor_distribution["mean"] = mean

  ones_indices = [i for (i, v) in enumerate(odor) if v]
  cov = np.zeros((n, n))
  for i in ones_indices:
    for j in ones_indices:
      if i == j:
        cov[i][j] = np.random.random()
      else:
        cov[i][j] = np.random.random()*1.5-0.75
        cov[j][i] = cov[i][j]


  odor_distribution["covariances"] = np.dot(cov, cov.T) # for psd covariance matrix

  distributions[id] = odor_distribution

In [4]:
# y_cov = distributions[1]["covariances"]
# y_mean = distributions[1]["mean"]
# # print(np.count_nonzero(x[5]))
# # print(np.count_nonzero(y_mean))
# # print(np.count_nonzero(y_cov))
# # np.min(np.real(np.linalg.eigvals(y_cov@y_cov.T)))
# # odor1 = np.random.multivariate_normal(distributions[1]["mean"], distributions[1]["covariances"], size=1)
# # print(len(x))
# diags = []
# for i in range(len(y_mean)):
#   if y_cov[i][i] != 0:
#     diags.append(y_cov[i][i])
# print(diags)
odor1 = np.random.multivariate_normal(distributions[2]["mean"], distributions[2]["covariances"], size=1)
odor1 = [v if abs(v) >= 0.01 else 0 for v in odor1[0]]
print(odor1)
for v in odor1:
  if v < 0:
    print(v)

[29.23448591172562, 33.82283519395624, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 33.35131915112555, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 34.66039135022656, 0, 0, 0, 0, 0, 0, 32.50270068215765, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 37.23123676074444, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26.688670931951485, 0, 0, 0, 0, 36.379410515224926, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32.266731445022764, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 31.625021637150716, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24.37147918842051, 0, 0, 0, 0, 37.845164562337644, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 39.431757231944026, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 33.688749029089976, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 31.02996122488951, 0, 0, 47.75349028448042, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 38.037940613069686, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Save distributions

In [5]:
with open('distributions.pkl', 'wb') as writefile: 
  pkl.dump(distributions, writefile)

In [2]:
with open('distributions.pkl','rb') as readfile:
  # ** x is a dictionary, with idx: [chems]
  distributions = pkl.load(readfile)

Generate training and validation data

In [6]:
# import csv
# def make_data(segment):
#   keys = list(distributions.keys())
#   segments = {
#     0: keys[0:63],
#     1: keys[63:124],
#     2: keys[124:187],
#     3: keys[187:249],
#     4: keys[249:312],
#     5: keys[312:374],
#     6: keys[374:436],
#     7: keys[436:]
#   }
#   samples_per_mixture = 30
#   for split in ['gaussian/training.csv', 'gaussian/validation.csv']:
#     with open(split, 'w') as writefile:
#       writer = csv.writer(writefile)
#       for id1 in segments[segment]:
#         for id2 in keys:
#           if id1 == id2:
#             continue
#           # for _ in range(samples_per_mixture):
#           odor1 = np.random.multivariate_normal(distributions[id1]["mean"], distributions[id1]["covariances"], size=samples_per_mixture)
#           odor2 = np.random.multivariate_normal(distributions[id2]["mean"], distributions[id2]["covariances"], size=samples_per_mixture)
#           mixture = odor1 + odor2
#           mixture = [[str([v if abs(v) >= 0.01 else 0 for v in mix]), (id1, id2)] for mix in mixture]
#           writer.writerows(mixture)
import csv
import numpy as np
import concurrent.futures
import threading

# Create a lock for synchronizing access to the CSV file
pickle_lock = threading.Lock()

# makes training data
def make_data(segment):
    keys = list(distributions.keys())
    segments = {
        0: keys[0:63],
        1: keys[63:124],
        2: keys[124:187],
        3: keys[187:249],
        4: keys[249:312],
        5: keys[312:374],
        6: keys[374:436],
        7: keys[436:]
    }
    samples_per_mixture = 30
    with pickle_lock:
      with open('gaussian/training.pkl', 'ab') as writefile:  # Open file in 'a' mode to append
        writer = csv.writer(writefile)
        for id1 in segments[segment]:
          for id2 in keys:
            if id1 == id2:
              continue
            # for _ in range(samples_per_mixture):
            odor1 = np.random.multivariate_normal(distributions[id1]["mean"], distributions[id1]["covariances"], size=samples_per_mixture)
            odor2 = np.random.multivariate_normal(distributions[id2]["mean"], distributions[id2]["covariances"], size=samples_per_mixture)
            mixture = odor1 + odor2
            mixture = [[str([v if abs(v) >= 0.01 else 0 for v in mix]), (id1, id2)] for mix in mixture]
            # Acquire a lock before writing to the file
            with pickle_lock:
              writer.writerows(mixture)


In [7]:
# List of arguments to pass to make_data function
args = list(range(0, 8))

# Create a ThreadPoolExecutor with maximum workers set to the number of available CPUs
with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:
    # Submit tasks for each argument in args to be executed concurrently
    futures = [executor.submit(make_data, arg) for arg in args]
    
    # Wait for all tasks to complete
    concurrent.futures.wait(futures)

In [ ]:
def make_validation():
  with open('gaussian/validation.csv', 'a', newline='') as writefile:  # Open file in 'a' mode to append
    writer = csv.writer(writefile)
    for _ in range(20000):
      keys = distributions.keys()
      id1 = np.random.choice(keys)
      id2 = np.random.choice(keys)
      while id1 == id2:
        id2 = np.random.choice(keys)
      odor1 = np.random.multivariate_normal(distributions[id1]["mean"], distributions[id1]["covariances"])
      odor2 = np.random.multivariate_normal(distributions[id2]["mean"], distributions[id2]["covariances"])
      mixture = odor1 + odor2
      mixture = [[str([v if abs(v) >= 0.01 else 0 for v in mix]), (id1, id2)] for mix in mixture]
      with csv_lock:
        writer.writerows(mixture)
